In [4]:
import torchvision.models as models
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
import foolbox.attacks as fa

from models.VGG16 import *

use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

if __name__ == "__main__":
    # instantiate a model
    model = VGG16().to(device)
    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
    fmodel = PyTorchModel(model, bounds=(-1, 1), preprocessing=preprocessing)
    
    state = torch.load('./models/model_150_89.pth')
    model.load_state_dict(state['model'])    
    model.eval()

    # get data and test the model
    # wrapping the tensors with ep.astensors is optional, but it allows
    # us to work with EagerPy tensors in the following
    images, labels = ep.astensors(*samples(fmodel, dataset="cifar10", batchsize=16, shape=(32, 32), bounds=(-1, 1)))
    print(accuracy(fmodel, images, labels))

    # apply the attack
    attack = fa.spatial_attack.SpatialAttack(
    max_translation=3,  # 6px so x in [x-6, x+6] and y in [y-6, y+6]
    num_translations=3,  # number of translations in x, y.
    max_rotation=5,  # +- rotation in degrees
    num_rotations=2,  # number of rotations
    # max total iterations = num_rotations * num_translations**2
    )
    
    xp_, _, success = attack(fmodel, images, labels)
    print(
        "attack success in specified rotation in translation bounds",
        success.numpy().astype(np.float32).mean() * 100,
        " %",
    )
    
    robust_accuracy = 1 - success.float32().mean(axis=-1)
    print(robust_accuracy)

/home/mlavrov/anaconda3/envs/meng/lib/python3.7/site-packages/foolbox/models/pytorch.py:37: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  "The PyTorch model is in training mode and therefore might"


0.875
attack success in specified rotation in translation bounds 68.75  %
PyTorchTensor(tensor(0.3125, device='cuda:0'))
